# Categorical feature encoding challenge - Baseline

In [31]:
import pandas as pd

data_path = '../input/cat-in-the-dat/'

train = pd.read_csv(data_path + 'train.csv', index_col='id')
test = pd.read_csv(data_path + 'test.csv', index_col='id')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col='id')

## 1. Feature engineering

In [32]:
# 1.1 데이터 합치기
all_data = pd.concat([train, test])
all_data.drop('target', axis=1, inplace=True)

# 1.2 원-핫 인코딩
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
all_data_encoded = encoder.fit_transform(all_data)

# 1.3 데이터 나누기
num_train = len(train)

X_train = all_data_encoded[:num_train]
X_test = all_data_encoded[num_train:]
y = train['target']

from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y, test_size=.1, stratify=y, random_state=10)

## 2. Model training

In [33]:
from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression(max_iter=1000, random_state=42)
logistic_model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42)

## 3. Evaluate model

In [34]:
from sklearn.metrics import roc_auc_score

y_valid_preds = logistic_model.predict_proba(X_valid)[:, 1]

roc_auc = roc_auc_score(y_valid, y_valid_preds)

print(f'검증 데이터 ROC AUC : {roc_auc:.4f}')

검증 데이터 ROC AUC : 0.7965


## 4. Prediction and Submit

In [36]:
y_preds = logistic_model.predict_proba(X_test)[:, 1]

submission['target'] = y_preds
submission.to_csv('submission.csv')